# JULIA MPI First Example: pi computaton

First step was to load MPI on my mac.  Seems mpich and openmpi are two reasonable choices
with probably no beginner's reason to prefer one over the other. <br>

I did  <t> brew install gcc </t> first to get the gcc compiler.  I ran into problems.  
The magic thing that told me what to do was <t> brew doctor </t>.  It wanted me to type
<t> xcode-select --install </t> and when I did, all was good.  I then typed
<t> brew install mpich </t> and mpi was just working.

My first example was to reproduce <a href="http://www.mcs.anl.gov/research/projects/mpi/tutorial/mpiexmpl/src/pi/C/solution.html">
the classic mypi </a> in the notebook

In [1]:
if !isdir(Pkg.dir("MPI"))  Pkg.add("MPI");   end

In [1]:
using MPI

In [46]:
m = MPIManager(np=4)

MPI.MPIManager(np=4,launched=false,mode=MPI_ON_WORKERS)

In [47]:
addprocs(m)
#@mpi_do m comm = MPI.COMM_WORLD

4-element Array{Int64,1}:
 4
 5
 6
 7

In [48]:
@mpi_do m comm = MPI.COMM_WORLD
#
# Enter number of intervals, and tell every processor
# Traditional MPI would do this with a BCAST
@mpi_do m n = 45

In [49]:
# Let's see if the processors got it
@mpi_do m println(n)

	From worker 7:	45
	From worker 6:	45
	From worker 4:	45
	From worker 5:	45


In [50]:
# my MPI id
@mpi_do m myid = MPI.Comm_rank(comm)
@mpi_do m println(myid)

	From worker 5:	1
	From worker 7:	3
	From worker 4:	0
	From worker 6:	2


In [51]:
# Get the number of processors
@mpi_do m np = MPI.Comm_size(comm)
@mpi_do m println(np)

	From worker 5:	4
	From worker 4:	4
	From worker 7:	4
	From worker 6:	4


Compute $\int_0^1 4/(1+x^2) dx= 4 atan(x)]_0^1$ which evaluates to $\pi$

In [52]:
using Interact

In [53]:
@time @mpi_do m begin
    n = 50_000_000
    comm = MPI.COMM_WORLD
    s = 0.0
    for i = MPI.Comm_rank(comm) + 1 : MPI.Comm_size(comm) : n 
        x = (i - .5)/n 
        s += 4/(1 + x^2) 
    end
    mypi = s/n
    our_π = MPI.Reduce(mypi, MPI.SUM, 0, comm)
    if myid==0
        println(our_π - π) 
    end
end

	From worker 4:	1.1146639167236572e-13
  3.296955 seconds (3.24 k allocations: 137.479 KB)


In [55]:
@fetchfrom 4 our_π, π

(3.1415926535899046,π = 3.1415926535897...)

In [56]:
function f_serial()
    n = 50_000_000
    h = 1/n
    our_π = 0.0
    for i = 0:h:1
        our_π += 4/(1 + i^2)
    end
    our_π*h
end

function f_serial2(n)
    our_π = 0.0
    for i = 1:n
        x = (i - 0.5)/n
        our_π += 4/(1 + x^2)
    end
    our_π/n
end

f_serial2 (generic function with 2 methods)

In [66]:
f_serial() #warmup
@time f_serial()
f_serial2(50_000_000) #warmup
@time f_serial2(50_000_000)

  0.321482 seconds (5 allocations: 176 bytes)
  0.408122 seconds (5 allocations: 176 bytes)


3.1415926535895617

In [60]:
@time f_par(50_000_000)

  0.467089 seconds (819 allocations: 59.922 KB)


3.141592653589621

In [61]:
function f_par(n)

 @mpi_do m begin
    comm = MPI.COMM_WORLD
        
       
    s = 0.0
    for i = MPI.Comm_rank(comm) + 1 : MPI.Comm_size(comm) : $n 
        x = (i - .5)/$n 
        s += 4/(1 + x^2) 
    end
    mypi = s/$n
    our_π = MPI.Reduce(mypi, MPI.SUM, 0, comm)
    #if myid==0
     #   println(our_π - π) 
   # end
end
@fetchfrom 2 our_π   
end

f_par (generic function with 1 method)

In [65]:
@mpi_do m function _pi_sum_par(n)
    comm = MPI.COMM_WORLD

    s = 0.0
    for i = MPI.Comm_rank(comm) + 1 : MPI.Comm_size(comm) : n
        x = (i - .5)/n 
        s += 4/(1 + x^2) 
    end
    mypi = s/n
    our_π = MPI.Reduce(mypi, MPI.SUM, 0, comm)
    return our_π
end
function f_par2(n)
    @mpi_do m _pi_sum_par($n)
    @fetchfrom 2 our_π
end
f_par(50_000_000) #warmup
@time f_par(50_000_000)
f_par2(50_000_000) #warmup
@time f_par2(50_000_000)

	From worker 5:	WARNING: Method definition _pi_sum_par(Any) in module Main at In[64]:2 overwritten at In[65]:2.
	From worker 7:	WARNING: Method definition _pi_sum_par(Any) in module Main at In[64]:2 overwritten at In[65]:2.
	From worker 6:	WARNING: Method definition _pi_sum_par(Any) in module Main at In[64]:2 overwritten at In[65]:2.
	From worker 4:	WARNING: Method definition _pi_sum_par(Any) in module Main at In[64]:2 overwritten at In[65]:2.


  0.411546 seconds (730 allocations: 53.656 KB)
  0.107494 seconds (725 allocations: 53.297 KB)


3.141592653589621

In [13]:
f(100)

9.739973446667229e7

In [14]:
f(1000)

9.739973446667229e7

In [15]:
f(10000)

9.739973446667229e7

In [ ]:
@manipulate for k=(3:10)
    f(10^k)
end

Interact.Options{:SelectionSlider,Int64}(Signal{Int64}(6, nactions=1),"k",6,"6",Interact.OptionDict(DataStructures.OrderedDict("3"=>3,"4"=>4,"5"=>5,"6"=>6,"7"=>7,"8"=>8,"9"=>9,"10"=>10),Dict(7=>"7",4=>"4",9=>"9",10=>"10",3=>"3",5=>"5",8=>"8",6=>"6")),Any[],Any[],true,"horizontal")

9.739973446667229e7

In [32]:
π

π = 3.1415926535897...

In [16]:
[f_par(10^k) for k=3:9]-π

7-element Array{Float64,1}:
 8.33333e-8 
 8.33339e-10
 8.30847e-12
 1.45661e-13
 1.91847e-13
 2.2915e-13 
 2.14051e-13

In [45]:
rmprocs(m)

:ok